<!-- File automatically generated using DocOnce (https://github.com/doconce/doconce/):
doconce format ipynb IntroductionNuMeODE.do.txt KODE=ja SPLITSEC=yes --skip_inline_comments --ipynb_disable_mpl_inline -->

<!-- KODE = "ja", "nei", default "ja" -->

# Numerical solution of ordinary differential equations: Euler's and Heun's method
**Anne Kværnø, André Massing**

If you want to have a nicer theme for your jupyter notebook,
download the [cascade stylesheet file calculus4N.css](https://www.math.ntnu.no/emner/TMA4130/2021h/lectures/calculus4N.css)
and execute the next cell:

In [ ]:
from IPython.core.display import HTML
def css_styling():
    try:
        fname = "calculus4N.css"
        with open(fname, "r") as f:
            styles = f.read()
            return HTML(styles)
    except FileNotFoundError:
        print(f"Could not find css file '{fname}'")

# Comment out next line and execute this cell to restore the default notebook style 
css_styling()

And of course we want to import the required modules.
As always, we start by importing the necessary modules:

In [ ]:
%matplotlib widget

import numpy as np
from numpy import pi
import matplotlib.pyplot as plt

# Use a funny plotting style
plt.xkcd()
newparams = {'figure.figsize': (6.0, 6.0), 'axes.grid': True,
             'lines.markersize': 8, 'lines.linewidth': 2,
             'font.size': 14}
plt.rcParams.update(newparams)

# Introduction: Whetting your appetite
The topic of this note is the numerical solution of systems of
ordinary differential equations (ODEs).  This has been discussed in
previous courses, see for instance the web page
[Differensialligninger](https://wiki.math.ntnu.no/tma4100/tema/differentialequations)
from Mathematics 1, as well as in Part 1 of this course, where the
Laplace transform was introduced as a tool to solve ODEs analytically.

Before we present the first numerical methods to solve ODEs, we quickly
look at a number of examples which hopefully will will serve as test examples
throughout this topic.

## Scalar first order ODEs
A scalar, first-order ODE is an equation on the form

$$
y'(t) = f(t,y(t)), \qquad y(t_0)=y_0,
$$

where $y'(t)=\frac{dy}{dt}$.
The *inital condition* $y(t_0)=y_0$ is required for a unique
solution. 

**Notice.**

It is common to use the term *initial value problem (IVP)* for an ODE
for which the inital value $y(t_0)=y_0$ is given, and we only are
interested in the solution for $t>t_0$. In these lecture notes, only
initial value problems are considered.

## Example 1: Population growth and decay processes

<div id="ode:exa:exponential"></div>
One of the simplest possible IVP is given by

<!-- Equation labels as ordinary links -->
<div id="ode:eq:exponential"></div>

$$
\begin{equation}
y'(t) = \lambda y(t), \quad y(t_0)=y_0.
\label{ode:eq:exponential} \tag{1}
\end{equation}
$$

In this case, the righ-hand side $f$ is simpy given by

$$
f(t, y) := \lambda y,
$$

so $f$ does not *explicitly* depend on $t$.

For $\lambda > 0$ this equation can present a simple model for the growth of
some population, e.g., cells, humans, animals, with unlimited resources
(food, space etc.). The constant $\lambda$ then corresponds to the
*growth rate* of the population.

Negative $\lambda < 0$
typically appear in decaying processes, e.g., the decay of a radioactive
isotopes, where $\lambda$ is then simply called the *decay constant*.

The analytical solution to ([1](#ode:eq:exponential)) is

<!-- Equation labels as ordinary links -->
<div id="ode:eq:exponential_sol"></div>

$$
\begin{equation}
y(t) = y_0 e^{\lambda(t-t_0)}
\label{ode:eq:exponential_sol} \tag{2}
\end{equation}
$$

and will serve us at several occasions as reference solution to assess
the accuracy of the numerical methods to be introduced.

## Example 2: Time-dependent coefficients

Given the ODE

$$
y'(t) = -2ty(t), \quad y(0) = y_0.
$$

for some given initial value $y_0$.

Now, the righ-hand side $f(t,y)$ is simpy given by

$$
f(t, y) := -2ty(t).
$$

The general solution of the ODE is

$$
y(t) = C e^{-t^2},
$$

where $C$ is a constant. To determine the constant $C$,
we use the initial condition $y(0) = y_0$
yielding the solution

$$
y(t) = y_0 e^{-t^2}.
$$

## Systems of ODEs
A system of $m$ ODEs are given by

$$
\begin{align*}
y_1' &= f_1(t,y_1,y_2,\dotsc,y_m), & y_1(t_0) &= y_{1,0} \\ 
y_2' &= f_2(t,y_1,y_2,\dotsc,y_m), & y_2(t_0) &= y_{2,0} \\ 
     & \vdots                      &          &\vdots    \\ 
y_m' &= f_m(t,y_1,y_2,\dotsc,y_m), & y_m(t_0) &= y_{m,0} \\ 
\end{align*}
$$

or more compactly by

$$
\mathbf{y}'(t) = \mathbf{f}(t, \mathbf{y}(t)),  \qquad \mathbf{y}(t_0) = \mathbf{y}_0
$$

where we use boldface to denote vectors in $\mathbb{R}^m$,

$$
\mathbf{y}(t) =
\left(
\begin{array}{c}
y_1(t) 
\\ y_2(t) 
\\ \vdots 
\\ y_m(t)
\end{array}
\right),
\qquad
\mathbf{f}(t,\mathbf{y}) =
\left(
\begin{array}{c}
f_1(t,y_1,y_2,\dotsc,y_m), 
\\ f_2(t,y_1,y_2,\dotsc,y_m), 
\\ \vdots 
\\ f_m(t,y_1,y_2,\dotsc,y_m),
\end{array}
\right),
\qquad
\mathbf{y}_0 =
\left(
\begin{array}{c}
y_{1,0} 
\\ y_{2,0} 
\\ \vdots 
\\ y_{m,0}
\end{array}
\right).
$$

## Example 3: Lotka-Volterra equation

<div id="ode:exa:lotka-volterra"></div>
The [Lotka-Volterra equation](https://en.wikipedia.org/wiki/Lotka-Volterra_equations) is
a system of two ODEs describing the interaction between preys and
predators over time. The system is given by

$$
\begin{align*}
y'(t) &= \alpha y(t) - \beta y(t) z(t) \\ 
z'(t) &= \delta y(t)z(t) - \gamma z(t)
\end{align*}
$$

where $x$ denotes time, $y(t)$ describes the population of preys and
$z(t)$ the population of predators.  The parameters $\alpha, \beta,
\delta$ and $\gamma$ depends on the populations to be modeled.

## Example 4: Spreading of diseases

<div id="ode:exa:fluspread"></div>
Motivated by the ongoing corona virus pandemic, we consider
a (simple!) model for the spreading of an infectious disease,
which goes under the name [SIR model](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model).

The SIR models divides the population into three
population classes, namely
S(t):
  :    
  number individuals  **susceptible** for infection,

I(t):
  :    
  number **infected** individuals, capable of transmitting the disease,

R(t):
  :    
  number  **removed** individuals who cannot be infected due death or to immunity after recovery

The model is of the spreading of a disease is based
on moving individual from $S$ to $I$ and then to $R$.
A short derivation can be found in [[LingeLangtangen2016]](#LingeLangtangen2016) (Ch. 4.2).
The final ODE system is given by

<!-- Equation labels as ordinary links -->
<div id="ode:eq:sir_model_s"></div>

$$
\begin{equation}
S' = - \beta S I
\label{ode:eq:sir_model_s} \tag{3}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="ode:eq:sir_model_i"></div>

$$
\begin{equation}  
I' = \beta S I - \gamma I
\label{ode:eq:sir_model_i} \tag{4}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="ode:eq:sir_model_r"></div>

$$
\begin{equation}  
R' = \gamma I,
\label{ode:eq:sir_model_r} \tag{5}
\end{equation}
$$

where $\beta$ denotes the infection rate, and $\gamma$ the removal rate.

## Higher order ODEs
An initial value  ODE of order $m$ is given by

$$
u^{(m)} = f(t,u,u',\dotsc,u^{(m-1)}), \qquad u(t_0)=u_0, \quad
u'(t_0)=u'_0,\quad  \dotsc, \quad u^{(m-1)}(t_0) = u^{(m-1)}_0.
$$

Here $u^{(1)} =u'$ and $u^{(m+1)}=\frac{du^{(m)}}{dx}$, for $m>0$.

## Example 5: Van der Pol's equation

<div id="ode:exa:vanderpol"></div>

[Van der Pol's equation](https://en.wikipedia.org/wiki/Van_der_Pol_oscillator)
is a second order differential equation, given by:

$$
u^{(2)} = \mu (1-u^2)u' - u, \qquad u(0)=u_0, \quad u'(0)=u'_0.
$$

where $\mu>0$ is some constant.  As initial values $u_0=2$ and
$u'_0=0$ are common choices.

Later in the note we will see how such equations can be rewritten as a
system of first order ODEs.  Systems of higher order ODEs can be
treated similarly.

# Euler's method
Now we turn to our first numerical method,
namely
[Euler's method](https://wiki.math.ntnu.no/tma4100/tema/differentialequations?&#numeriske_losninger),
known from Mathematics 1.
We quickly review two alternative derivations,
namely one based on *numerical differentiation*
and one on *numerical integration*.

## Derivation of Euler's method
Euler's method is the simplest example of a so-called
**one step method (OSM)**.
Given the IVP

$$
y'(t) = f(t,y(t)), \qquad y(t_0)=y_0,
$$

and some final time $T$,
we want to compute to an approximation of $y(t)$
on $[t_0, T]$.

We start from $t_0$ and choose some (usually small) time step size
$\tau_0$ and set the new time $t_1 = t_0 + \tau_0$. The goal is to
compute a value $y_1$ serving as approximation of $y(t_1)$.

To do so, we Taylor expand the exact (but unknown) solution
$y(t_0+\tau)$ around $x_0$:

$$
y(t_0+\tau) = y(t_0) + \tau y'(t_0) + \frac{1}{2}\tau^2 y''(t_0) + \dotsm.
$$

Assume the step size $\tau$ to be small, such that the solution is
dominated by the first two terms.
In that case, these can be used as
the numerical approximation in the next step:

$$
y(t_0+\tau) \approx  y(t_0) + \tau y'(t_0) = y_0 + \tau f(t_0, y_0)
$$

giving

$$
y_1 = y_0 + \tau_0 f(t_0,y_0).
$$

Now we can repeat this procedure and choose the next
(possibly different) time
step $\tau_1$ and compute a numerical approximation $y_2$
for $y(t)$ at $t_2 = t_1 + \tau_1$ by setting

$$
y_2 = y_1 + \tau_1 f(t_1,y_1).
$$

The idea is to repeat this procedure until we reached the
final time $T$ resulting in the following

**Recipe: Euler's method.**

Given a function $f(t,y)$ and an initial value $(t_0,y_0)$.
* Set $t = t_0$.

* $\texttt{while } t < T$:

  * Choose step-size $\tau_k$

  * $\displaystyle y_{k+1} := y_{k} + \tau_k f(t_k, y_k)$ 

  * $t_{k+1}:=t_k+\tau_k$

  * $t := t_{k+1}$

So we can think of the Euler method as a method
which approximates the continuous but unknown solution
$y(t): [t_0, T] \to \mathbb{R}$
by a discrete function
$y_{\Delta}:\{t_0, t_1, \ldots, t_{N_t}\}$
such that $y_{\Delta}(t_k) := y_k \approx y(t_k)$.

How to choose $\tau_k$? The simplest possibility
is to set a maximum number of steps $N_{\mathrm{max}} = N_t$ and then
to chose a *constant time step* $\tau = (T-t_0)/N_{\mathrm{max}}$
resulting in $N_{\mathrm{max}}+1$ equidistributed points.
Later we will also learn, how to choose the
*time step adaptively*, depending on the
solution's behavior.

**Numerical solution between the nodes.**

At first we have only an approximation of $y(t)$
at the $N_t +1 $ nodes $y_{\Delta}:\{t_0, t_1, \ldots, t_{N_t}\}$.
If we want to evaluate the numerical solution between the
nodes, a natural idea is to extend the discrete solution
linearly between each pair of time nodes $t_{k}, t_{k+1}$.
This is compatible with the way the numerical solution can
be plotted, namely by connected each pair
$(t_k, y_k)$ and $(t_{k+1}, y_{k+1})$ with straight lines.

Also, in order to compute an approximation
at the next point $t_{k+1}$,
Euler's method only needs to know $f$, $\tau_k$
and the solution $y_k$ at the *current* point $t_k$,
but not at earlier points $t_{k-1}, t_{k-2}, \ldots$
Thus Euler's method
is an prototype of a so-called **One Step Method (OSM)**.
We will formalize this concept later.

**Interpretation: Euler's method via forward difference operators.**

After rearranging terms,
we can also interpret the computation of
an approximation $y_1 \approx y(t_1)$
as replacing the
derivative $y'(t_0) = f(t_0, y_0)$ with a **forward difference operator**

$$
f(t_0,y_0) = y'(t_0) \approx
 \dfrac{y(t_1) - y(t_0)}{\tau}
$$

Thus *Euler's method replace the differential quotient
by a difference quotient.*

**Alternative derivation via numerical integration.**
First we recall that for a function $f: [a,b] \to \mathbb{R}$, we can
approximate its integral $\int_a^b f(t) {\,\mathrm{d}t}$ by
a *very simple*  quadrature rule of the form

<!-- Equation labels as ordinary links -->
<div id="ode:ex:rectangular_qr"></div>

$$
\begin{equation}
\int_a^b f(t) {\,\mathrm{d}t} \approx (b-a) f(a).
\label{ode:ex:rectangular_qr} \tag{6}
\end{equation}
$$

**Notice.**

Recall from course module on numerical integration that the degree of
exactness of the previous quadrature rule is 0.

Turning to our IVP, we know formally integrate
the ODE $y'(t) = f(t, y(t))$ on the time
interval $I_k = [t_k, t_{k+1}]$
and then applying the quadrature rule ([6](#ode:ex:rectangular_qr))
leading to

$$
y(t_{k+1}) - y(t_k)
=
\int_{t_k}^{t_{k+1}} y'(t) {\,\mathrm{d}t}
=
\int_{t_k}^{t_{k+1}} f(t,y(t)) {\,\mathrm{d}t}
\approx
\underbrace{(t_{k+1}-t_{k})}_{\tau_k}f(t_k, y(t_k))
$$

Sorting terms gives us back Euler's method

$$
y(t_{k+1}) \approx
y(t_k) + \tau_k f(t_k, y(t_k)).
$$

## Implementation of Euler's method
Euler's method can be implemented in only a few lines of code:

In [ ]:
def explicit_euler(y0, t0, T, f, Nmax):
    ys = [y0]
    ts = [t0]
    dt = (T - t0)/Nmax
    while(ts[-1] < T):
        t, y = ts[-1], ys[-1]  
        ys.append(y + dt*f(t, y))
        ts.append(t + dt)
    return (np.array(ts), np.array(ys))

Let's test Euler's method with the simple IVP given in 
Example [Example 1: Population growth and decay processes](#ode:exa:exponential) and compute the numerical
solution for $N_{\mathrm{max}} = 4, 8, 16.$

In [ ]:
t0, T = 0, 1
y0 = 1
lam = 1
Nmax = 5

# rhs of IVP
f = lambda t,y: lam*y

# Exact solution to compare against
y_ex = lambda t: y0*np.exp(lam*(t-t0))

# Solve numerically and plot solutions
plt.rcParams["figure.figsize"] = (10,4)
Nmax_list = [4, 8, 16]
fig, axes = plt.subplots(1,len(Nmax_list))

for i in range(len(Nmax_list)):
    ts, ys_eul = explicit_euler(y0, t0, T, f, Nmax=Nmax_list[i])
    ys_ex = y_ex(ts)
    axes[i].plot(ts, ys_ex, 'bo-')
    axes[i].plot(ts, ys_eul, 'rx-')
    axes[i].legend(["$y_{\mathrm{ex}}$", "$y_{\mathrm{euler}}$" ])

Plot the solution for various $N_t$,
say $N_t = 4, 8, 16$ against the exact solution given in 
Example [Example 1: Population growth and decay processes](#ode:exa:exponential).

## Exercise 1: Error study for the Euler's method
<div id="ode:exe:euler_error_study"></div>

We observed that the more we decrease the constant step size $\tau$ (or increase $N_{\mathrm{max}}$),
the closer the numerical solution gets to the exact solution.

Now we ask you to quantify this. More precisely,
write some code to compute the error

$$
\max_{i \in \{0, \ldots, N_{\mathrm{max}}\}} |y(t_i) - y_i|
$$

for $N_{\mathrm{max}} = 4, 8, 16, 32, 64, 128, 256$.
How does the error reduces if you double the number of points?

In [ ]:
# Insert your code here.

<!-- --- begin solution of exercise --- -->
**Solution.**

In [ ]:
def error_study(y0, t0, T, f, Nmax_list, solver, y_ex):
    max_errs = []
    for Nmax in Nmax_list:
        ts, ys = solver(y0, t0, T, f, Nmax)
        ys_ex = y_ex(ts)
        errors = ys - ys_ex
        max_errs.append(np.abs(errors).max())
        print("For Nmax = {:3}, max ||y(t_i) - y_i||= {:.3e}".format(Nmax,max_errs[-1]))
    print("The computed error reduction rates are")
    max_errs = np.array(max_errs)
    print(max_errs[:-1]/max_errs[1:])

Nmax_list = [4, 8, 16, 32, 64, 128, 256]
error_study(y0, t0, T, f, Nmax_list, explicit_euler, y_ex)

We observe that for a uniform/constant time step, if we double the number
of steps/reduce the time step by $2$, the error reduces also by a factor of $2$.

<!-- --- end solution of exercise --- -->

# Heun's method
Before we start to look at more exciting examples,
we will derive a one step method which is more accurate then Euler's
method.
Note that Euler's method can be interpreted as being based on a quadrature
rule with degree of exactness equal to $0$. 
Let's try to use a better quadrature rule!

Again, we start from the *exact representation*,
but this time we use the trapezoidal rule,
which has degree of exactness equal to $1$,
yielding

$$
\begin{align*}
y(t_{k+1}) - y(t_k)
&=
\int_{t_k}^{t_{k+1}} f(t,y(t)) {\,\mathrm{d}t}
\approx
\dfrac{\tau_k}{2}
\left(
f(t_{k+1}, y(t_{k+1})
+
f(t_{k}, y(t_{k})
\right)
\end{align*}
$$

This suggest to consider the scheme

$$
\begin{align*}
y_{k+1} - y_k
&=
\dfrac{\tau_k}{2}
\left(
f(t_{k+1}, y_{k+1})
+
f(t_{k}, y({k})
\right)
\end{align*}
$$

But note that starting from $y_k$, we cannot immediately compute $y_{k+1}$
as it appears also in the expression $f(t_{k+1}, y_{k+1})$!
This is an example of an **implicit method** known
as *Crank-Nicolson method* or *implicit trapezoidal rule*. We will return to this method
later when we consider numerical methods for time-dependent heat equation.

To turn this scheme into an explicit scheme,
the idea is now to replace $y_{k+1}$ appearing in $f$ with an explicit Euler
step:

$$
\begin{align*}
y_{k+1}
&=
y_k +
\dfrac{\tau_k}{2}
\left(
f\bigl(t_{k+1}, y_{k}+\tau_k f(t_k, y_k)\bigr)
+
f(t_{k}, y_k)
\right).
\end{align*}
$$

Observe that we have now nested evaluations of $f$. This can be best
arranged by computing the nested expression in stages, first
the inner one and then the outer one.
This leads to the following recipe.

**Recipe: Heun's method.**

Given a function $f(t,y)$ and an initial value $(t_0,y_0)$.
* Set $t = t_0$.

* $\texttt{while } t < T$:

  * Choose $\tau_k$

  * Compute stage $k_1 := f(t_k, y_k)$

  * Compute stage $k_2 := f(t_k+\tau_k, y_k+\tau_k k_1)$

  * $\displaystyle y_{k+1} := y_{k} + \tfrac{\tau_k}{2}(k_1 + k_2)$ 

  * $t_{k+1}:=t_k+\tau_k$

  * $t := t_{k+1}$

The solver function `heun` can be implemented in a similar manner as the `explicit_euler`:

In [ ]:
def heun(y0, t0, T, f, Nmax):
    ys = [y0]
    ts = [t0]
    dt = (T - t0)/Nmax
    while(ts[-1] < T):
        t, y = ts[-1], ys[-1]
        k1 = f(t,y)
        k2 = f(t+dt, y+dt*k1)
        ys.append(y + 0.5*dt*(k1+k2))
        ts.append(t + dt)
    return (np.array(ts), np.array(ys))

## Exercise 2: Comparing Heun with Euler
<div id="exe:heun_vs_euler"></div>

**a)**
Redo the Example [Example 1: Population growth and decay processes](#ode:exa:exponential) with Heun, and plot
both the exact solution, $y_{eul}$ and $y_{heun}$
for $N_t = 4, 8, 16$.

In [ ]:
# Insert code here.

<!-- --- begin solution of exercise --- -->
**Solution.**

In [ ]:
Nmax_list = [4, 8, 16]
fig, axes = plt.subplots(1,len(Nmax_list))

for i in range(len(Nmax_list)):

    ts, ys_eul = explicit_euler(y0, t0, T, f, Nmax=Nmax_list[i])
    ts, ys_heun = heun(y0, t0, T, f, Nmax=Nmax_list[i])
    ys_ex = y_ex(ts)
    axes[i].plot(ts, ys_ex, 'bo-')
    axes[i].plot(ts, ys_eul, 'rx-')
    axes[i].plot(ts, ys_heun, 'g*-')
    axes[i].legend(["$y_{\mathrm{ex}}$", "$y_{\mathrm{euler}}$", "$y_{\mathrm{heun}}$"])

<!-- --- end solution of exercise --- -->

**b)**
Redo [Exercise 1: Error study for the Euler's method](#ode:exe:euler_error_study) with Heun.

In [ ]:
# Insert code here.

<!-- --- begin solution of exercise --- -->
**Solution.**

In [ ]:
Nmax_list = [4, 8, 16, 32, 64, 128, 256]
error_study(y0, t0, T, f, Nmax_list, heun, y_ex)

<!-- --- end solution of exercise --- -->

**Discussion.**
We observe that for Heun's method and a uniform/constant time step, if we double of the number
of steps/reduce the time step by $2$, the error reduces by a factor of $4$.

Moreover, for Euler's method we need 256 uniform time steps corresponding to a $\tau = 1/256$ to
obtain an approximation error of $5.290e-03$ while for Heun, we only need 8 steps with $\tau = 1/8$
to obtain a similar approximation error of $6.441e-03$!

Comparing the complexity of Euler's method with Heun's method, we see that Heun's method
uses on only twice as many function evaluations (one for $k_1$ and one for $k_2$) as Euler's method.
So we can say that 8 steps with Heun is roughly as expensive as 16 steps with Euler.
So clearly, Heun's method should be favored over Euler's method due to its higher accuracy and
improved error reduction rate.

# Applying Heun's and Euler's method

## Example 6: The Lotka-Volterra equation revisited

Solve the Lotka-Volterra equation

$$
\begin{align*}
y'(t) &= \alpha y(t) - \beta y(t) z(t) \\ 
z'(t) &= \delta y(t)z(t) - \gamma z(t)
\end{align*}
$$

In this example, use the parameters and initial values

$$
\alpha=2,\quad \beta=1, \quad \delta=0.5,\quad \gamma=1, \qquad y_{1,0}=2,
\quad y_{2,0} = 0.5.
$$

* Use Euler's method to solve the equation over the interval $[0,20]$, and use $\tau=0.02$. Try also other step sizes, e.g. $\tau=0.1$ and $\tau=0.002$.

* Redo the last example with Heun's method and compare the solutions generated by Euler's and Heun's method. In particular compare the number of steps you need to perform with Euler vs Heun to obtain visually the "same" solution. (That is, you measure the difference of the two numerical solutions in the "eyeball norm".)

**NB!** For this example, the exact solution is not known.  What is
known is that the solutions are periodic and positive. Is this the
case here?  Check for different values of $\tau$.

In [ ]:
# Reset plotting parameters
plt.rcParams.update({'figure.figsize': (12,6)})

In [ ]:
# Define rhs
def lotka_volterra(t, y):
    # Set parameters
    alpha, beta, delta, gamma = 2, 1, 0.5, 1  
    # Define rhs of ODE
    dy = np.array([alpha*y[0]-beta*y[0]*y[1], 
                delta*y[0]*y[1]-gamma*y[1]])
    return dy

t0, T = 0, 20            # Integration interval
y0 = np.array([2, 0.5])  # Initital values

# Reset plotting parameters
plt.cla()
plt.rcParams.update({'figure.figsize': (12,6)})
plt.figure()

# Solve the equation with Euler and plot results
tau = 0.02
Nmax = int(20/tau)
print("Nmax = {:4}".format(Nmax))
ts, ys_eul = explicit_euler(y0, t0, T, lotka_volterra, Nmax)
plt.plot(ts, ys_eul)

# Solve the equation with Heun and add them to plot
tau = 0.02
Nmax = int(20/tau)
print("Nmax = {:4}".format(Nmax))
ts, ys_heun = heun(y0, t0, T, lotka_volterra, Nmax)
plt.plot(ts, ys_heun)

plt.xlabel('t')
plt.legend(['$y(t)$ - Euler', '$z(t)$ - Euler', '$y(t)$ - Heun', '$z(t)$ - Heun'],
        loc="upper right")

## Higher order ODEs
How can we numerically solve higher order ODEs
using, e.g., Euler's or Heun's method?

Given the $m$-th order ODE

$$
u^{(m)}(t) = f\big(t, u(t), u'(x), \dotsc, u^{(m-1)}\big).
$$

For a unique solution, we assume that the initial values

$$
u(t_0), u'(t_0), u''(t_0), \dotsc, u^{(m-1)}(t_0)
$$

are known.

Such equations can be written as a system of first order ODEs by the
following trick:
Let

$$
y_1(t) = u(t), \quad y_2(t) = u'(t), \quad
y_3(t) = u^{(2)}(t), \quad \dotsc \quad, y_{m}(t) = u^{(m-1)}(t)
$$

such that

$$
\begin{align*}
  y_1' &= y_2, & y_1(a) &= u(a) \\ 
  y_2' &= y_3, & y_2(a) &= u'(a) \\ 
       & \vdots  && \vdots\\ 
  y_{m-1}' &= y_m, & y_{m-1}(a) &= u^{(m-2)}(a) \\ 
  y_m' &= f(t, y_1, y_2, \ldots, y_{m-1},y_m), & y_m(a) &= u^{(m-1)}(a)
\end{align*}
$$

which is nothing but a system of first order ODEs, and can be solved numerically
exactly as before.

## Exercise 3: Numerical solution of Van der Pol's equation
<div id="ode:exer:vanderpol_numerics"></div>

Recalling Example [Example 5: Van der Pol's equation](#ode:exa:vanderpol), the Van der Pol oscillator
is described by the second order differential equation

$$
u'' = \mu (1-u^2)u' - u, \qquad u(0)=u_0, \quad u'(0)=u_0'.
$$

It can be rewritten as a system of first order ODEs:

$$
\begin{align*}
y_1' &= y_2, & y_1(0) &= u_0,  \\ 
y_2' &= \mu(1-y_1^2)y_2 - y_1, & y_2(0) &= u_0'.
\end{align*}
$$

**a)**
Let  $\mu=2$, $u(0)=2$ and $u'(0)=0$ and solve the equation over the interval
$[0,20]$, using the explicit Euler and $\tau=0.1$. Play with different
step sizes, and maybe also with different values of $\mu$.

**b)**
Repeat the previous numerical experiment with Heun's method.
Try to compare the number of steps you need to perform
with Euler vs Heun to obtain visually the "same" solution.
(That is, you measure the difference of the two numerical solutions
in the "eyeball norm".)

In [ ]:
# Insert code here.

<!-- --- begin solution of exercise --- -->
**Solution.**

In [ ]:
# Define the ODE
def f(t, y):
    mu = 2
    dy = np.array([y[1],
                mu*(1-y[0]**2)*y[1]-y[0] ])
    return dy

# Set initial time, stop time and initial value
t0, T = 0, 20
y0 = np.array([2,0])

# Create new figure
plt.figure()

# Solve the equation using Euler and plot
tau = 0.1
Nmax = int(20/tau)
print("Nmax = {:4}".format(Nmax))
ts, ys_eul  = explicit_euler(y0, t0, T, f, Nmax)
plt.plot(ts,ys_eul);

# Solve the equation using Heun
tau = 0.1
Nmax = int(20/tau)
print("Nmax = {:4}".format(Nmax))
ts, ys_heun = heun(y0, t0, T, f, Nmax)
plt.plot(ts,ys_heun);

plt.xlabel('$t$')
plt.title('Van der Pols ligning')
plt.legend(['$y1$ - Euler','$y2$ - Euler', '$y1$ - Heun','$y2$ - Heun'], loc='upper right');

<!-- --- end solution of exercise --- -->

# Bibliography
1. <div id="LingeLangtangen2016"></div> **S. Linge and H. P. Langtangen**.  *Programming for Computations - Python*, 1 edition, Springer, 2016.